**Run these cells before training or inferring from a fine-tuned model**

In [ ]:
# Mounts Google drive to access the folders used for I/0
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import re
import os

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [ ]:
!pip install transformers[torch]
# Sometimes after installing the accelerate library, it is still not recognised. If that happens, restart the runtime and rerun the pip command
!pip install accelerate -U

**Training**

In [ ]:
def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

In [ ]:
# Read documents from the directory
train_directory = '/content/drive/MyDrive/FinetuningGPT2/training_v2.txt'
text_data = read_txt(train_directory)
text_data = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters


In [ ]:
with open("/content/drive/MyDrive/FinetuningGPT2/train.txt", "w") as f:
    f.write(text_data)

In [ ]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [ ]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [ ]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [ ]:

train_file_path = "/content/drive/MyDrive/FinetuningGPT2/train.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/FinetuningGPT2/model'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 50.0
save_steps = 50000

In [ ]:
# Train
# Fine-tuning the GPT-2 model and saving the resulting model in output_dir. If there is not a suitable GPU in the system, this will not work.
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


**Inference**

In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [ ]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

# Loads the model from the specified path and infers from it. If there is not a suitable GPU in the system, this will not work.
def generate_text(model_path, sequence, max_length):

    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
        # num_beams = 5,
        # no_repeat_ngram_size=2,
        early_stopping=True,
        num_return_sequences=1
    )
    return (tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [ ]:
# Specify the path of the fine-tuned model
model2_path = "/content/drive/MyDrive/FinetuningGPT2/model"
# specify the max_length parameter for the model
max_len = 50

In [ ]:
# File path to example input on google drive
inputFile = "/content/drive/MyDrive/openAI_input/outputAdaptiveSystem.csv"

In [ ]:
# List to store the cleaned triples
triples = []

with open(inputFile) as input:
    for line in input:
      elements = line.strip().split(",")
      # Remove unused number
      elements.pop(-1)
      # Clean the elements
      new_elem = []
      for word in elements:
        new_elem.append(word.replace("food_galmat_1.7.", ""))
      # Convert cleaned list into a tuple
      new_elem = tuple(new_elem)
      triples.append(new_elem)
print(triples)

[('Bakso', 'hasIngredient', 'Noodle', '0.3'), ('Bakso', 'hasIngredient', 'Vermicelli', '0.3'), ('Bakso', 'hasIngredient', 'Celery', '0.3'), ('Bakso', 'isEatenInCountry', 'NationwideInIndonesia', '0.3'), ('Bakso', 'isEatenInCountry', 'ChineseCuisine', '0.3'), ('Bakso', 'isEatenInRegion', 'NeighboringSoutheastAsianCountries', '0.3'), ('Bakso', 'isEatenInRegion', 'NationwideInIndonesia', '0.3'), ('BarnyCakes', 'hasIngredient', 'SpongeCake', '-0.732'), ('BarnyCakes', 'isEatenInCountry', 'France', '-0.732'), ('BarnyCakes', 'isCreatedBy', 'MondelezInternational', '-0.732'), ('BarnyCakes', 'hasDishVariation', 'Chocolate', '-0.732'), ('BarnyCakes', 'hasDishVariation', 'Apple', '-0.732'), ('BaconExplosion', 'hasIngredient', 'Sausage', '-0.268'), ('BaconExplosion', 'hasIngredient', 'Bacon', '-0.268'), ('BaconExplosion', 'isEatenInCountry', 'UnitedStates', '-0.268'), ('BaconExplosion', 'hasMainIngredients', 'Sausage', '-0.268'), ('BaconExplosion', 'hasMainIngredients', 'Bacon', '-0.268'), ('Bacon

In [ ]:
# Array to save each description
sentences = []
for triple in triples:
  subject, predicate, obj, learner_ability = triple
  sequence = f"[Q] How to describe ({subject}, {predicate}, {obj})?"
  validText = ""
  # A for loop to attempt to get a valid description out of the model
  for i in range(0,5):
    genText = generate_text(model2_path, sequence, max_len)
    validText = genText[genText.find("[A]")+4:genText.find(".")+1]
    if (len(validText) == 0):
      i = i - 1
      continue
    if ((validText.find(subject)!= -1) and (validText.find(obj)!= -1)):
      break
  sentences.append(validText)

print("\n".join(sentences))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


A dish that contains tapioca is Bakso.
Vermicelli is an ingredient in Bakso.
Celery is a member of the genus Bakso.
Bakso is eaten in Indonesia, where the leader is Joko Widodo.
Bakso is made from wheat and vegetables.
Bakso is a variant of an Italian sauce.
Bakso is a type of noodles made nationwide in Indonesia.
Sponge cakes can be made with beetroot or minced meat fillings.
Barny cakes can be made in France.
Mondelez International is the creator of Barny cakes.
Barny cakes can be chocolate flavoured.
Barny cakes can be made with apple.
A sausage is used when making a Bacon Explosion.
A bacon explosion originates from the country of origin of the United States.

Bacon Explosion comes from the country of the United States, where two of the main ingredients are sausage and beef.
Bacon Explosion comes from the United Kingdom.
Bacon Explosion comes from the Kansas city metro area in the U.
A main course is a BLT.


In [ ]:
# Save the output of the fine-tuned GPT2 model in a text file
outputPath = "/content/drive/MyDrive/GPT2_output/GPT2output_3.txt"
with open(outputPath, "w") as file:
        file.write("\n".join(sentences))